In [2]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import yaml
import pandas as pd
import numpy as np

In [ ]:
# Load the YAML configuration file
with open('/Users/MiladM-Dev/Documents/1PhD/project-1-N450/project-1.1-gendata/ReMaster/Scripts/xml_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

df = pd.read_csv(config["csv"]["FilePath"], encoding="utf-8", sep = ";")
df.columns = df.iloc[0] # Remove the first row
df.drop(df.index[0], inplace=True)
df.reset_index(drop=True, inplace=True)
  # Drop the first row after setting it as header

df

,Trial number,Paramters,Infection Rate (β),Incubation Rate (σ),Recovery Rate (γ),Sampling rate,S,E,I,R,Sample
0,1,SEIR_pruning,2.10,83,0.14,850,740,0,1,0,1
1,2,SEIR_pruning,2.10,83,0.14,850,100,0,1,0,1
2,3,SEIR_pruning,2.10,83,0.14,470,100,0,1,0,1
3,4,SEIR_pruning,2.10,83,0.14,80,100,0,1,0,1
4,5,SEIR_pruning,2.10,83,0.14,21,100,0,1,0,1
5,6,SEIR_pruning,2.10,83,0.14,57,100,0,1,0,1


convert df to array

In [110]:
arr = df.to_numpy()

for i, row in enumerate(arr):
    trial_number, paramters, InfectionRate, IncubationRate, RecoveryRate, SamplingRate,	S,	E,	I,	R,	Sample = row
    print(f"Run {i+1}: trial_number={trial_number}, paramters={paramters}")

Run 1: trial_number=1, paramters=SEIR_pruning
Run 2: trial_number=2, paramters=SEIR_pruning
Run 3: trial_number=3, paramters=SEIR_pruning
Run 4: trial_number=4, paramters=SEIR_pruning
Run 5: trial_number=5, paramters=SEIR_pruning
Run 6: trial_number=6, paramters=SEIR_pruning


In [9]:
# Load the YAML configuration file
with open('/Users/MiladM-Dev/Documents/1PhD/project-1-N450/project-1.1-gendata/ReMaster/Scripts/xml_config.yaml', 'r') as file:
    config = yaml.safe_load(file)
class generate_xml_files:
    def __init__(self, config):
        self.config = config

    def read_csv(self):
        self.df = pd.read_csv(self.config["csv"]["FilePath"], encoding="utf-8", sep = ";")
        self.df.columns = self.df.iloc[0] # Remove the first row
        self.df.drop(self.df.index[0], inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        return self.df


    def create_xml(self):
        # convert DataFrame to numpy array
        arr = self.df.to_numpy()
        for i, self.row in enumerate(arr):
            self.trial_number, self.paramters, self.InfectionRate, self.IncubationRate, self.RecoveryRate, self.SamplingRate, self.S, self.E, self.I, self.R, self.Sample = self.row
            
            print(f"Run {i+1}: trial_number={self.trial_number}, paramters={self.paramters}, InfectionRate={self.InfectionRate}, "
                  f"IncubationRate={self.IncubationRate}, RecoveryRate={self.RecoveryRate}, "
                  f"SamplingRate={self.SamplingRate}, S={self.S}, E={self.E}, I={self.I}, R={self.R}, Sample={self.Sample}")
            
            # Create the root element
            root = ET.Element("beast", version=self.config['beast']['version'], namespace=self.config['beast']['namespace'])

            # Add a run element with sub-elements
            run = ET.SubElement(root, "run", spec=self.config['run']["spec"], nSims= self.config['run']['nSims'])

            simulate = ET.SubElement(run, "simulate", spec=self.config['run']["simulate"]["spec"], id=self.config['run']["simulate"]["id"])
            trajectory = ET.SubElement(simulate, "trajectory", spec="StochasticTrajectory", id="traj")

            # Add population elements
            ET.SubElement(trajectory, "population", spec="RealParameter", id="S", value=self.S)
            ET.SubElement(trajectory, "population", spec="RealParameter", id="E", value=self.E)
            ET.SubElement(trajectory, "population", spec="RealParameter", id="I", value=self.I)

            # Add samplePopulation elements
            ET.SubElement(trajectory, "samplePopulation", spec="RealParameter", id="R", value= self.R)  
            ET.SubElement(trajectory, "samplePopulation", spec="RealParameter", id="sample", value= self.Sample)

            # Add reaction elements
            ET.SubElement(trajectory, "reaction", spec="Reaction", rate=self.InfectionRate).text = "I + S -> I + E"
            ET.SubElement(trajectory, "reaction", spec="Reaction", rate=self.IncubationRate).text = "E -> I"
            ET.SubElement(trajectory, "reaction", spec="Reaction", rate=self.RecoveryRate).text = "I -> R"
            ET.SubElement(trajectory, "reaction", spec="Reaction", rate=self.SamplingRate).text = "I -> R + sample"

            # Add logger elements
            logger1 = ET.SubElement(run, "logger", spec="Logger", fileName="$(filebase).traj")
            ET.SubElement(logger1, "log", idref="traj")

            logger2 = ET.SubElement(run, "logger", spec="Logger", mode="tree", fileName="$(filebase).full.trees")
            log2 = ET.SubElement(logger2, "log", spec="TypedTreeLogger")
            ET.SubElement(log2, "tree", spec="PrunedTree", samplePops="R", simulatedTree="@tree")

            logger3 = ET.SubElement(run, "logger", spec="Logger", mode="tree", fileName="$(filebase).sampled.trees")
            log3 = ET.SubElement(logger3, "log", spec="TypedTreeLogger")
            ET.SubElement(log3, "tree", spec="PrunedTree", samplePops="sample", simulatedTree="@tree")


            # Create the ElementTree object
            tree = ET.ElementTree(root)

            # Write to XML file
            tree.write(f"/Users/MiladM-Dev/Documents/1PhD/project-1-N450/project-1.1-gendata/ReMaster/SEIR_pruning/Trial1/{self.paramters}_{i+1}.xml", encoding="utf-8", xml_declaration=True)

            #xml_str = ET.tostring(root, encoding='utf-8')
            #pretty_xml = minidom.parseString(xml_str).toprettyxml(indent="  ")
            #print(pretty_xml)
            

    def excute(self):
        self.read_csv()

        self.create_xml()

xml_files = generate_xml_files(config)
xml_files.excute()

Run 1: trial_number=1, paramters=SEIR_pruning, InfectionRate=2.10, IncubationRate=83, RecoveryRate=0.14, SamplingRate=850, S=740, E=0, I=1, R=0, Sample=1
Run 2: trial_number=2, paramters=SEIR_pruning, InfectionRate=2.10, IncubationRate=83, RecoveryRate=0.14, SamplingRate=850, S=100, E=0, I=1, R=0, Sample=1
Run 3: trial_number=3, paramters=SEIR_pruning, InfectionRate=2.10, IncubationRate=83, RecoveryRate=0.14, SamplingRate=470, S=100, E=0, I=1, R=0, Sample=1
Run 4: trial_number=4, paramters=SEIR_pruning, InfectionRate=2.10, IncubationRate=83, RecoveryRate=0.14, SamplingRate=80, S=100, E=0, I=1, R=0, Sample=1
Run 5: trial_number=5, paramters=SEIR_pruning, InfectionRate=2.10, IncubationRate=83, RecoveryRate=0.14, SamplingRate=21, S=100, E=0, I=1, R=0, Sample=1
Run 6: trial_number=6, paramters=SEIR_pruning, InfectionRate=2.10, IncubationRate=83, RecoveryRate=0.14, SamplingRate=57, S=100, E=0, I=1, R=0, Sample=1


In [87]:
import numpy as np
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5, 6],
    'B': [10, 20, 30, 40, 50, 60],
    'C': [100, 200, 300, 400, 500, 600],
    'D': [7, 8, 9, 10, 11, 12],
    'E': [70, 80, 90, 100, 110, 120],
    'F': [700, 800, 900, 1000, 1100, 1200],
    'G': [0, 1, 0, 1, 0, 1]
})
arr = df.to_numpy()

for i, row in enumerate(arr):
    a, b, c, d, e, f, g = row
    print(f"Run {i+1}: a={a}, b={b}, c={c}, d={d}, e={e}, f={f}, g={g}")

df

Run 1: a=1, b=10, c=100, d=7, e=70, f=700, g=0
Run 2: a=2, b=20, c=200, d=8, e=80, f=800, g=1
Run 3: a=3, b=30, c=300, d=9, e=90, f=900, g=0
Run 4: a=4, b=40, c=400, d=10, e=100, f=1000, g=1
Run 5: a=5, b=50, c=500, d=11, e=110, f=1100, g=0
Run 6: a=6, b=60, c=600, d=12, e=120, f=1200, g=1


,A,B,C,D,E,F,G
0,1,10,100,7,70,700,0
1,2,20,200,8,80,800,1
2,3,30,300,9,90,900,0
3,4,40,400,10,100,1000,1
4,5,50,500,11,110,1100,0
5,6,60,600,12,120,1200,1


In [ ]:
# Create the root element
root = ET.Element("beast", version=config['beast']['version'], namespace=config['beast']['namespace'])

# Add a run element with sub-elements
run = ET.SubElement(root, "run", spec=config['run']["spec"], nSims= config['run']['nSims'])

simulate = ET.SubElement(run, "simulate", spec=config['run']["simulate"]["spec"], id=config['run']["simulate"]["id"])
trajectory = ET.SubElement(simulate, "trajectory", spec="StochasticTrajectory", id="traj")

# Add population elements
ET.SubElement(trajectory, "population", spec="RealParameter", id="S", value=config['run']["simulate"]["trajectory"]["population"]["spec_S"])
ET.SubElement(trajectory, "population", spec="RealParameter", id="E", value=config['run']["simulate"]["trajectory"]["population"]["spec_E"])
ET.SubElement(trajectory, "population", spec="RealParameter", id="I", value=config['run']["simulate"]["trajectory"]["population"]["spec_I"])

# Add samplePopulation elements
ET.SubElement(trajectory, "samplePopulation", spec="RealParameter", id="R", value= config['run']["simulate"]["trajectory"]["samplePopulation"]["spec_R"])  
ET.SubElement(trajectory, "samplePopulation", spec="RealParameter", id="sample", value= config['run']["simulate"]["trajectory"]["samplePopulation"]["spec_sample"])

# Add reaction elements
ET.SubElement(trajectory, "reaction", spec="Reaction", rate=config['run']["simulate"]["trajectory"]["reaction"]["spec_I_E_rate"]).text = "I + S -> I + E"
ET.SubElement(trajectory, "reaction", spec="Reaction", rate=config['run']["simulate"]["trajectory"]["reaction"]["spec_E_I_rate"]).text = "E -> I"
ET.SubElement(trajectory, "reaction", spec="Reaction", rate=config['run']["simulate"]["trajectory"]["reaction"]["spec_R_rate"]).text = "I -> R"
ET.SubElement(trajectory, "reaction", spec="Reaction", rate=config['run']["simulate"]["trajectory"]["reaction"]["spec_sampling_rate"]).text = "I -> R + sample"

# Add logger elements
logger1 = ET.SubElement(run, "logger", spec="Logger", fileName="$(filebase).traj")
ET.SubElement(logger1, "log", idref="traj")

logger2 = ET.SubElement(run, "logger", spec="Logger", mode="tree", fileName="$(filebase).full.trees")
log2 = ET.SubElement(logger2, "log", spec="TypedTreeLogger")
ET.SubElement(log2, "tree", spec="PrunedTree", samplePops="R", simulatedTree="@tree")

logger3 = ET.SubElement(run, "logger", spec="Logger", mode="tree", fileName="$(filebase).sampled.trees")
log3 = ET.SubElement(logger3, "log", spec="TypedTreeLogger")
ET.SubElement(log3, "tree", spec="PrunedTree", samplePops="sample", simulatedTree="@tree")


# Create the ElementTree object
tree = ET.ElementTree(root)

# Write to XML file
tree.write("/Users/MiladM-Dev/Documents/1PhD/project-1-N450/project-1.1-gendata/ReMaster/library.xml", encoding="utf-8", xml_declaration=True)


# Convert to string and pretty print
xml_str = ET.tostring(root, encoding='utf-8')
pretty_xml = minidom.parseString(xml_str).toprettyxml(indent="  ")

# Print the XML tree to console
print(pretty_xml)
# Optionally, write the XML to a file